In [7]:
import cv2
import os
from pathlib import Path
from matplotlib import pyplot as plt
import math
import numpy as np
from tqdm import tqdm

In [8]:
path = Path("Dataset_TP3")
videos_paths = [p for p in path.iterdir() if p.is_file()]

videos = {}
for file in videos_paths:
    vid = cv2.VideoCapture(file)
    if not vid.isOpened():
        print("Error: Could not open video.")
        continue
    videos[file] = vid
    break

In [9]:
m_x = [[-1, 0, 1], [-2, 0, 2], [-1, 0, 1]]
m_y = [[-1, -2, -1], [0, 0, 0],[1, 2, 1], ]

edges = {}

for video_path in videos:
    video  = videos[video_path]
    frame_width = int(video.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))

    fps = video.get(cv2.CAP_PROP_FPS)
    frame_count = int(video.get(cv2.CAP_PROP_FRAME_COUNT))
    edges_lst = []

    for frame_idx in tqdm(range(frame_count)):
        ret, frame = video.read()
        if not ret:
            print("Error: Could not read frame.")
            break

        gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        edged_frame = np.zeros_like(gray_frame)

        for i in range(1, frame_height - 1):
            for j in range(1, frame_width - 1):
                sum_x = 0
                sum_y = 0
                for k in range(-1, 2):
                    for l in range(-1, 2):
                        pixel = float(gray_frame[i + k, j + l])
                        sum_x += pixel * m_x[k + 1][l + 1]
                        sum_y += pixel * m_y[k + 1][l + 1]
                magnitude = math.sqrt(sum_x ** 2 + sum_y ** 2)
                edged_frame[i, j] = min(255, int(magnitude))

        edges_lst.append(edged_frame)
    edges[video_path] = edges_lst
        # cv2.imshow('Edged Frame', edged_frame)
        # if cv2.waitKey(int(1000 / fps)) & 0xFF == ord('q'):
        #     break

  0%|          | 0/5047 [00:00<?, ?it/s]

  0%|          | 19/5047 [00:22<1:39:00,  1.18s/it]


KeyboardInterrupt: 